In [1]:
#Yingshu Wang 100614552
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from os import getcwd

#Load Model

In [2]:
#Using InceptionV3  Model
from tensorflow.keras.applications.inception_v3 import InceptionV3


In [3]:
translation = ["dog",  "horse",  "elephant", 
             "butterfly", "chicken",  "cat", 
              "cow",  "sheep",  "squirrel"]

In [4]:

pre_trained_model = InceptionV3(input_shape = (275,275,3),
                                include_top = False, # Leave out the last fully connected layer
                                weights = 'imagenet')

#all the layers non trainable
for layer in pre_trained_model.layers:
  layer.trainable = False

In [5]:
from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer 
x = layers.GlobalAveragePooling2D()(pre_trained_model.output)

# 1024 units and do ReLU activation
x = layers.Dense(1024, activation = 'relu')(x)

# Add a final sigmoid layer for classification
x = layers.Dense  (10, activation = 'sigmoid')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(loss='categorical_crossentropy',
                  optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 275, 275, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 137, 137, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 137, 137, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 137, 137, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [6]:
#External Data from https://www.kaggle.com/alessiocorrado99/animals10/notebooks?datasetId=59760&sortBy=dateRun&tab=profile


#Import Data Dic and preprocess

In [7]:
import os 
#Under google drive file
folder = os.path.join('/content/drive/MyDrive/raw-img')



In [8]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,
      validation_split=0.33,
      horizontal_flip=True,
      fill_mode='nearest')

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1/255)

In [9]:
from tensorflow.keras.preprocessing import image_dataset_from_directory
BATCH_SIZE = 32
IMG_SIZE = 275

train_ds =  train_datagen.flow_from_directory(directory=folder, 
                                        subset='training',
                                        target_size=(IMG_SIZE, IMG_SIZE),
                                        batch_size=BATCH_SIZE)

Found 17546 images belonging to 10 classes.


In [10]:
valid_ds =  train_datagen.flow_from_directory(directory=folder, 
                                        subset='validation',
                                        target_size=(IMG_SIZE, IMG_SIZE),
                                        batch_size=BATCH_SIZE)

Found 8633 images belonging to 10 classes.


#Train Model

In [11]:
# Define a Callback class that stops training once accuracy reaches 99.0%
# Ref : https://github.com/sahil301290/Deep-Learning/blob/master/Lab%20Codes/07.%20Transfer%20Learning/Lab_7_Transfer_Learning.ipynb

In [12]:
history = model.fit(
    train_ds,
    #steps_per_epoch = 75, #smaller value, quicker
    validation_data = valid_ds, 
    #validation_steps = int(?),
    epochs = 2) #higher number of epochs, higher accuracy/ In this case, because the epochs is small, lead to the low accuracy

Epoch 1/2
549/549 [==============================] - 978s 2s/step - loss: 0.3408 - accuracy: 0.9097 - val_loss: 0.1179 - val_accuracy: 0.9686
Epoch 2/2
549/549 [==============================] - 121s 220ms/step - loss: 0.1005 - accuracy: 0.9684 - val_loss: 0.1124 - val_accuracy: 0.9710


In [23]:
from tensorflow.keras import preprocessing
from tensorflow import expand_dims
from tensorflow.nn import softmax
img = preprocessing.image.load_img('sample_image.jpg', target_size=(275, 275, 3))
import numpy as np

img_array = preprocessing.image.img_to_array(img)
img_array = expand_dims(img_array, 0)/255

predictions = model.predict(img_array)

print(translation[np.argmax(predictions)])

elephant
